In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.interpolate import griddata
from utilities.common_functions import clean_dataframe, create_historical_dataframe
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Exploring with station 44013
coordinates = [42.346, -70.651]
nc_file = 'ERA5/Processed/era5_2019_-83_-65_37_49.nc'

In [ ]:
def load_era5(nc_file):
    dataset = xr.open_dataset(nc_file)
    # Convert the dataset to a pandas DataFrame
    df = dataset.to_dataframe()
    # Reset index to move time, latitude, and longitude from index to columns
    df_reset = df.reset_index()
    return df_reset

In [ ]:
def find_four_closest_points(df_reset, coordinates):
    # Compute the Euclidean distance between each point and the target
    df_reset["distance"] = np.sqrt((df_reset['latitude'] - coordinates[0])**2 + (df_reset['longitude'] - coordinates[1])**2)
    # sort the dataframe by time and distance
    df_sorted = df_reset.sort_values(by=['time', 'distance'])
    
    # keep the four first rows for each time
    df_sorted = df_sorted.groupby('time').head(4)
    return df_sorted


In [ ]:
def linear_interpolation(dataset, coordinates):
    points = dataset[['latitude', 'longitude']][:4].values
    interpolated_df_value = pd.DataFrame(columns=['time','u_interp', 'v_interp', 'ssr_interp', 't2m_interp', 'd2m_interp'])
    nb_values = len(dataset)
    
    # create a matrix with the batch of 4 points with the values of u
    u_values = dataset['u10'].values
    v_values = dataset['v10'].values
    ssr_values = dataset['ssr'].values
    t2m_values = dataset['t2m'].values
    d2m_values = dataset['d2m'].values
    u_matrix = u_values[:nb_values // 4 * 4].reshape(-1, 4).T
    v_matrix = v_values[:nb_values // 4 * 4].reshape(-1, 4).T
    ssr_matrix = ssr_values[:nb_values // 4 * 4].reshape(-1, 4).T
    t2m_matrix = t2m_values[:nb_values // 4 * 4].reshape(-1, 4).T
    d2m_matrix = d2m_values[:nb_values // 4 * 4].reshape(-1, 4).T
    
    
    # Define an interpolation function
    def interpolate_column(col):
        return griddata(points, col, (coordinates[0], coordinates[1]), method='linear')

    # Apply the interpolation function to each column of the matrices
    interpolated_df_value['u_interp'] = np.apply_along_axis(interpolate_column, 0, u_matrix)
    interpolated_df_value['v_interp'] = np.apply_along_axis(interpolate_column, 0, v_matrix)
    interpolated_df_value['ssr_interp'] = np.apply_along_axis(interpolate_column, 0, ssr_matrix)
    interpolated_df_value['t2m_interp'] = np.apply_along_axis(interpolate_column, 0, t2m_matrix)
    interpolated_df_value['d2m_interp'] = np.apply_along_axis(interpolate_column, 0, d2m_matrix)
    interpolated_df_value['time'] = dataset['time'].unique()
    
    
    
    

    return interpolated_df_value
    

In [ ]:
def time_interpolating_dataset(interpolate_df):
    # interpolate between two times
    start_time = interpolate_df['time'].min()
    end_time = interpolate_df['time'].max()

    # Create a time range starting at the first occurrence of `00:50`, and repeating every hour
    new_times = pd.date_range(start=start_time + pd.Timedelta(minutes=50), 
                            end=end_time + pd.Timedelta(hours=1), 
                            freq='h')

    interpolate_df.set_index('time', inplace=True)
    # Reindex the original dataframe to include new times
    df_reindexed = interpolate_df.reindex(interpolate_df.index.union(new_times))


    # Interpolate the missing values
    df_time_interpolated = df_reindexed.interpolate(method='time')
    
    return df_time_interpolated
    

In [ ]:
def process_buoy_data(station_name, year):
    # Load the buoy data
    dataframe = pd.DataFrame()
    dataframe = create_historical_dataframe(station_name, year, dataframe)
    dataframe = clean_dataframe(dataframe)
    dataframe['u_velocity'] = dataframe['WDIR'].apply(lambda x: -np.sin(np.radians(x))) * dataframe['WSPD']
    dataframe['v_velocity'] = dataframe['WDIR'].apply(lambda x: -np.cos(np.radians(x))) * dataframe['WSPD']
    dataframe['ATMP'] = dataframe['ATMP'] + 273.15
    dataframe['DEWP'] = dataframe['DEWP'] + 273.15
    return dataframe

In [ ]:
dataset = load_era5(nc_file)

In [ ]:
dataset = find_four_closest_points(dataset, coordinates)

In [ ]:
interpolate_df = linear_interpolation(dataset, coordinates)

In [ ]:
df_time_interpolated = time_interpolating_dataset(interpolate_df)

In [ ]:
buoy_dataframe = process_buoy_data('44013', 2019)

In [ ]:
comparative_dataset = pd.merge(buoy_dataframe, df_time_interpolated, left_index=True, right_index=True, how='inner')

In [ ]:
comparative_dataset

In [ ]:
# Correlation between ERA5 velocity and buoy velocity
correlation_u = comparative_dataset['u_velocity'].corr(comparative_dataset['u_interp'])
correlation_v = comparative_dataset['v_velocity'].corr(comparative_dataset['v_interp'])
correlation_dewp = comparative_dataset['DEWP'].corr(comparative_dataset['d2m_interp'])
correlation_t2m = comparative_dataset['ATMP'].corr(comparative_dataset['t2m_interp'])


In [ ]:
def visualize_correlation(dataset, variable_1, variable_2, variable_type, correlation):
    
    
    # Visualization 1: Scatter plot with regression line
    plt.figure(figsize=(8, 6))
    sns.regplot(x=variable_1, y=variable_2, data=dataset)
    plt.title(f'Scatter Plot of {variable_type} (ERA5 vs Buoy) - Correlation: {correlation:.2f}')
    plt.xlabel(f'{variable_1} (ERA5)')
    plt.ylabel(f'{variable_2} (Buoy)')
    plt.show()

    # Visualization 2: Time-series comparison
    plt.figure(figsize=(10, 6))
    plt.plot(dataset.index, dataset[variable_1], label=f'ERA5 {variable_1}', marker='o')
    plt.plot(dataset.index, dataset[variable_2], label=f'Buoy {variable_2}', marker='x')
    plt.title(f'Time Series of {variable_2} vs {variable_1} (ERA5 vs Buoy)')
    plt.xlabel('Time')
    plt.ylabel(f'{variable_type}')
    plt.legend()
    plt.show()

In [ ]:
visualize_correlation(comparative_dataset, 'u_interp', 'u_velocity', 'Velocity', correlation_u)
visualize_correlation(comparative_dataset, 'v_interp', 'v_velocity', 'Velocity', correlation_v)
visualize_correlation(comparative_dataset, 'd2m_interp', 'DEWP', 'Temperature', correlation_dewp)
visualize_correlation(comparative_dataset, 't2m_interp', 'ATMP', 'Temperature', correlation_t2m)


In [ ]:
file = 'A01_met_all_4c4d_7d6d_ac7a.nc'

dataframe = load_era5(file)

In [ ]:
dataframe.columns

In [ ]:
dataframe


In [ ]:
dataframe.drop(columns=['wind_2_gust', 'wind_2_gust_qc',
       'wind_2_speed', 'wind_2_speed_qc', 'wind_2_direction',
       'wind_2_direction_qc'], inplace=True)

In [ ]:
dataframe